## [TFRS Quick Start](https://www.tensorflow.org/recommenders/examples/quickstart) 

### Import Necessary Libraries

In [1]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [2]:
# Ratings data.
ratings = tfds.load('movielens/100k-ratings', split="train")
# Features of all the available movies.
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])

Shuffling and writing examples to /Users/shovon/tensorflow_datasets/movielens/100k-ratings/0.1.0.incomplete2KVCGE/movielens-train.tfrecord


Dataset movielens downloaded and prepared to /Users/shovon/tensorflow_datasets/movielens/100k-ratings/0.1.0. Subsequent calls will reuse this data.


Shuffling and writing examples to /Users/shovon/tensorflow_datasets/movielens/100k-movies/0.1.0.incompleteOXD7BC/movielens-train.tfrecord


Dataset movielens downloaded and prepared to /Users/shovon/tensorflow_datasets/movielens/100k-movies/0.1.0. Subsequent calls will reuse this data.


In [3]:
user_ids_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

In [4]:
class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(self, user_model: tf.keras.Model, movie_model: tf.keras.Model,task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.movie_model = movie_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["movie_title"])

    return self.task(user_embeddings, movie_embeddings)

In [5]:
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
  )
)

In [6]:
# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=3)

Epoch 1/3


25/25 [==============================] - 6s 260ms/step - factorized_top_k/top_1_categorical_accuracy: 1.8000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0025 - factorized_top_k/top_10_categorical_accuracy: 0.0066 - factorized_top_k/top_50_categorical_accuracy: 0.0527 - factorized_top_k/top_100_categorical_accuracy: 0.1142 - loss: 33073.0030 - regularization_loss: 0.0000e+00 - total_loss: 33073.0030
Epoch 2/3
25/25 [==============================] - 7s 262ms/step - factorized_top_k/top_1_categorical_accuracy: 4.7000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0055 - factorized_top_k/top_10_categorical_accuracy: 0.0153 - factorized_top_k/top_50_categorical_accuracy: 0.1077 - factorized_top_k/top_100_categorical_accuracy: 0.2136 - loss: 31011.9509 - regularization_loss: 0.0000e+00 - total_loss: 31011.9509
Epoch 3/3
25/25 [==============================] - 7s 265ms/step - factorized_top_k/top_1_categorical_accuracy: 7.0000e-04 - factorized_top_k/top_5_categorical_accuracy

In [9]:
# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index(movies.batch(100).map(model.movie_model), movies)

# Get some recommendations.
_, titles = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {titles[0, :3]}")

Top 3 recommendations for user 42: [b'Rent-a-Kid (1995)' b'House Arrest (1996)' b'Just Cause (1995)']
